<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#概要" data-toc-modified-id="概要-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>概要</a></span></li><li><span><a href="#2019-01-04" data-toc-modified-id="2019-01-04-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2019-01-04</a></span><ul class="toc-item"><li><span><a href="#Two-Sigma:気になったカーネル" data-toc-modified-id="Two-Sigma:気になったカーネル-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Two Sigma:気になったカーネル</a></span><ul class="toc-item"><li><span><a href="#Create-Data-Pipeline-and-LGBClassifierCV(score:0.65811)" data-toc-modified-id="Create-Data-Pipeline-and-LGBClassifierCV(score:0.65811)-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Create Data Pipeline and LGBClassifierCV(score:0.65811)</a></span></li></ul></li></ul></li></ul></div>

# 概要
Kaggleに関連する調査研究を書き溜めてゆくノートである。

# 2019-01-04
## Two Sigma:気になったカーネル

### Create Data Pipeline and LGBClassifierCV(score:0.65811)
以下のカーネルが気になっている。  
https://www.kaggle.com/aditya1702/create-data-pipeline-and-lgbclassifiercv  

これまで公開されてきたカーネルの要点を消化して、よくまとめているカーネルである。  
注目点は、  
* きちんとクラスを作っている
* 指数平滑移動平均など時系列特徴量(time-lagged-feature)の追加  
* パイプライン処理の導入
   ```python
        data_pipeline = Pipeline([
            ('quant_features_generator', AddingQuantFeatures()),
            ('news_preprocessor', PreprocessingNews()),
            ('news_unstacker', UnstackingNews()),
            ('news_grouping', GroupingNewsWithAggregations()),
            ('lgb', LGBClassifierCV(cv = 5,
                                    perform_random_search = False,
                                    perform_bayes_search = False,
                                    use_train_test_split = False,
                                    use_full_data = True,
                                    use_kfold_split = False)
            )
        ])

        data_pipeline.fit([market_train, news_train])
    ```

* かなり簡素なツリー構造。詳細はこちら。
  ```python
            self.lgb_optimal_params = {'learning_rate': 0.05, 
                                       'num_leaves': 12, 
                                       'n_estimators': 100, 
                                       'min_child_samples': 70, 
                                       'colsample_bytree': 0.9, 
                                       'subsample': 0.9, 
                                       'reg_alpha': 0.8, 
                                       'reg_lambda': 0.2, 
                                       'n_jobs': 3}
  
  ```
    
    * num_leaves が 12 とかなり極端に小さな値になっている。

* Post-Process で新しいパターンを発見した。  
  平均値を引いている。どんな分布になるのだろうか？
  ```python
          preds = pipeline.predict([market_obs_df, news_obs_df])
          preds = (preds - np.mean(preds))
  ```